<a href="https://colab.research.google.com/github/EltonWolf/GoogleEarthPythonBootcampAPI-/blob/main/Calculating_MODISNDVI_Egypt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee

In [2]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=L2mFl4eILe-Sdph-C6FV5K-71imp9ryLmWscLR4Lw-k&tc=b9hpep09cLqEC8sVKv24jOClqyqIpTTZ_WVyILI0cr0&cc=i9byzgaVeP-82bowO4I9LDfRfy-7MbtVtssseYecEk8

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXlCiqVgux9k5aIpnd9-9A1YISDMmzB3oI9LGprT7xJGOFSMcfb4PyE

Successfully saved authorization token.


In [30]:
#Load countries data
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
Ethiopia = countries.filter(ee.Filter.eq('country_na', 'Egypt'))

In [31]:
#MODIS image

#Load MODIS image
img = ee.Image('MODIS/006/MOD09GA/2012_03_09')

#Use the normalizedDifference function
ndvi = img.normalizedDifference(['sur_refl_b02', 'sur_refl_b01'])

In [33]:
# Make a 'palette': a list of hex strings.
palette = [
    'FFFFFF',
    'CE7E45',
    'DF923D',
    'F1B555',
    'FCD163',
    '99B718',

    '74A901',
    '66A000',
    '529400',
    '3E8601',
    '207401',
    '056201',

    '004C00',
    '023B01',
    '012E01',
    '011D01',
    '011301'
]

In [37]:

# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
my_map = folium.Map(location=[30.033, 31.233], zoom_start=5)

# Add the layer to the map object.
my_map.add_ee_layer(ndvi.clip(Ethiopia), {'min': 0, 'max': 1, 'palette': palette}, 'NDVI')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)